Server Operations Using Python's Psycopg2¶

In [1]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=localhost port=5432 dbname=cloud_user user=cloud_user password=cloud_user"

Create Table

In [2]:
create_table_query = '''CREATE TABLE banksim (
    step integer,
    customer varchar(25),
    age varchar(25),
    gender varchar(25),
    zipcodeOri varchar(25),
    merchant varchar(25),
    zipMerchant varchar(25),
    category varchar(25),
    amount numeric,
    fraud integer
); '''

try:
    #make connect db
    cnx = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cnx.cursor()

    # Send sql query to request
    cur.execute(create_table_query)
    records = cnx.commit()

except (Exception, psycopg2.Error) as error :
    print("Error while connecting to PostgreSQL", error)

finally:
    #closing database connection
    if(cnx):
        cur.close()
        cnx.close()
        print("PostgreSQL connection is closed")

print(f'Record:\n {records}')

PostgreSQL connection is closed
Record:
 None


Add The Data to Table

In [4]:
try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)
    
    # Create a cursor to db
    cur = cxn.cursor()
    
    with open('./bs140513_032310.csv', 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'banksim', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("tips table populated")

PostgreSQL connection is closed
tips table populated


Selecting Data From a Server

Use .fetchall() with LIMIT or TOP(#)
 - LIMIT works for most databases, but does not work with SQL Server
 - TOP (#) is used in place of LIMIT on SQL Server

In [14]:
def db_server_fetch(sql_query):
    try:
        # Make connection db
        cnx = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cnx.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cur.fetchall()

    except (Exception, psycopg2.Error) as error :
        print("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection
        if(cnx):
            cur.close()
            cnx.close()
            print("PostgreSQL connection is closed")
        return records

In [28]:
select_query = '''SELECT * FROM banksim LIMIT 5;'''

records = db_server_fetch(select_query)
print(records)

PostgreSQL connection is closed
[(0, 'C927282395', '4', 'F', 'M85975013', 'es_food', Decimal('66.52'), 0), (0, 'C352968107', '2', 'M', 'M348934600', 'es_transportation', Decimal('39.68'), 0), (0, 'C210584188', '4', 'M', 'M732195782', 'es_travel', Decimal('1993.68'), 0), (0, 'C927766810', '3', 'F', 'M348934600', 'es_transportation', Decimal('2.74'), 0), (0, 'C608547225', '2', 'F', 'M855959430', 'es_hyper', Decimal('76.51'), 0)]


In [29]:
records

[(0, 'C927282395', '4', 'F', 'M85975013', 'es_food', Decimal('66.52'), 0),
 (0,
  'C352968107',
  '2',
  'M',
  'M348934600',
  'es_transportation',
  Decimal('39.68'),
  0),
 (0, 'C210584188', '4', 'M', 'M732195782', 'es_travel', Decimal('1993.68'), 0),
 (0,
  'C927766810',
  '3',
  'F',
  'M348934600',
  'es_transportation',
  Decimal('2.74'),
  0),
 (0, 'C608547225', '2', 'F', 'M855959430', 'es_hyper', Decimal('76.51'), 0)]

Delete quotes from colomn


In [21]:
def remove_quotes_from_column():
    try:
        # Make connection db
        cnx = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cnx.cursor()

        # SQL command to remove quotes from multiple columns
        update_query = """
            UPDATE banksim
            SET 
                customer = REPLACE(customer, '''', ''),
                age = REPLACE(age, '''', ''),
                gender = REPLACE(gender, '''', ''),
                zipcodeOri = REPLACE(zipcodeOri, '''', ''),
                merchant = REPLACE(merchant, '''', ''),
                zipMerchant = REPLACE(zipMerchant, '''', ''),
                category = REPLACE(category, '''', '')
            WHERE 
                customer LIKE '%''%' OR
                age LIKE '%''%' OR
                gender LIKE '%''%' OR
                zipcodeOri LIKE '%''%' OR
                merchant LIKE '%''%' OR
                zipMerchant LIKE '%''%' OR
                category LIKE '%''%';
        """
        # Send sql query to request
        cur.execute(update_query)

        # Commit the changes to the database
        cnx.commit()

    except (Exception, psycopg2.Error) as error :
        print("Error while connecting to PostgreSQL", error)  

    finally:
        #closing database connection
        if(cnx):
            cur.close()
            cnx.close()
            print("Quotes successfully removed from multiple columns.")  

# Call the function to remove quotes from multiple columns
remove_quotes_from_column()

Quotes successfully removed from multiple columns.


Delete colomn zipcodeori dan zipmerchant karena nilainya sama semua

In [27]:
def drop_columns():
    try:
        # Make connection db
        cnx = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cnx.cursor()

        # SQL command to drop columns from the table
        drop_query = """
            ALTER TABLE banksim
            DROP COLUMN IF EXISTS zipcodeOri,
            DROP COLUMN IF EXISTS zipMerchant;
        """

        # Send sql query to request
        cur.execute(drop_query)

        # Commit the changes to the database
        cnx.commit()

    except (Exception, psycopg2.Error) as error :
        print("Error while connecting to PostgreSQL", error)  

    finally:
        #closing database connection
        if(cnx):
            cur.close()
            cnx.close()
            print("Columns 'zipcodeOri' and 'zipMerchant' dropped successfully.")

drop_columns()
        

Columns 'zipcodeOri' and 'zipMerchant' dropped successfully.


In [31]:
df = pd.DataFrame(records, columns=["step","customer","age","gender","merchant","category","amount","fraud"])

# Print the DataFrame
df

,step,customer,age,gender,merchant,category,amount,fraud
0,0,C927282395,4,F,M85975013,es_food,66.52,0
1,0,C352968107,2,M,M348934600,es_transportation,39.68,0
2,0,C210584188,4,M,M732195782,es_travel,1993.68,0
3,0,C927766810,3,F,M348934600,es_transportation,2.74,0
4,0,C608547225,2,F,M855959430,es_hyper,76.51,0


Setelah data ditransform langkah berikutnya adalah memindahkan data dari postgresql ke gcs dengan langkah pernama membuat file csv dari data yang sudah ditransform dengan code berikut

COPY (SELECT * FROM banksim) TO './banksim.csv' WITH CSV HEADER;
